In [1]:
import pandas as pd
import re
import os
import numpy as np
import emoji
import string
import nltk
from googletrans import Translator
from string import punctuation
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, LSTM, Conv1D, MaxPooling1D, Bidirectional

Using TensorFlow backend.


In [2]:
os.chdir('Downloads\AllProductReviews.csv')
reviews = pd.read_csv('AllProductReviews.csv')
reviews.head()

,ReviewTitle,ReviewBody,ReviewStar,Product
0,Honest review of an edm music lover\n,No doubt it has a great bass and to a great ex...,3,boAt Rockerz 255
1,Unreliable earphones with high cost\n,"This earphones are unreliable, i bought it be...",1,boAt Rockerz 255
2,Really good and durable.\n,"i bought itfor 999,I purchased it second time,...",4,boAt Rockerz 255
3,stopped working in just 14 days\n,Its sound quality is adorable. overall it was ...,1,boAt Rockerz 255
4,Just Awesome Wireless Headphone under 1000...😉\n,Its Awesome... Good sound quality & 8-9 hrs ba...,5,boAt Rockerz 255


In [3]:
title = reviews['ReviewTitle']
body = reviews['ReviewBody']
star = reviews['ReviewStar']

## Data Cleaning

In [4]:
#Combine the reviewTitle and ReviewBody columns into one
reviews['review'] = reviews['ReviewTitle'] + reviews['ReviewBody']

In [5]:
complete_review = reviews['review']

In [6]:
word = set(nltk.corpus.words.words())
sw = set(stopwords.words('english'))

In [7]:
def clean(text):
    text = text.strip()
    text = text.lower()
    text = text.split()
    text = ' '.join([string for string in text if not string in sw])
    text = re.sub(r'(?<=[.,])(?=[^\s])', r' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation+'0123456789'))
    text = ' '.join(text.split())
    text = ''.join(string for string in text if string not in emoji.UNICODE_EMOJI)
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in word)
    text = ' '.join([word for word in text.split() if len(word) > 2])
    return text
coll = complete_review.map(lambda x: clean(x))

In [8]:
coll.head()

0    honest review music lover doubt great bass gre...
1    unreliable high cost unreliable bought days me...
2    really good durable bought second time gifted ...
3    stopped working days sound quality adorable ov...
4    awesome wireless headphone awesome good sound ...
Name: review, dtype: object

## Keras Model Building

In [9]:
#Convert text into tokens with identitcal sequence length
max_seq_length = 100
vocab_size = 4000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(coll)
sequences = tokenizer.texts_to_sequences(coll)
x_train = pad_sequences(sequences, maxlen=max_seq_length, padding='post', truncating='post')

In [10]:
y_vector = reviews['ReviewStar'] - 1
y_train = to_categorical(y_vector, num_classes=5)

In [11]:
#Create the model architecture
desired_embedding_size = 100
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=desired_embedding_size,
                    input_length = max_seq_length))
model.add(Flatten())
model.add(Dropout(rate=(0.6)))
model.add(Dense(32, activation='tanh'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adagrad', loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train, y_train, validation_split=0.3, epochs=10)

W0905 15:45:36.302284 14968 deprecation_wrapper.py:119] From C:\Users\danie\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0905 15:45:36.355823 14968 deprecation_wrapper.py:119] From C:\Users\danie\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 15:45:36.588410 14968 deprecation_wrapper.py:119] From C:\Users\danie\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0905 15:45:36.642844 14968 deprecation_wrapper.py:119] From C:\Users\danie\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0905 15:45:36.661822 14968 deprecation.py:506

Train on 10035 samples, validate on 4302 samples
Epoch 1/10
10035/10035 [==============================] - 8s 777us/step - loss: 1.1332 - acc: 0.5389 - val_loss: 0.9647 - val_acc: 0.6390
Epoch 2/10
10035/10035 [==============================] - 12s 1ms/step - loss: 0.9061 - acc: 0.6349 - val_loss: 0.9162 - val_acc: 0.6536
Epoch 3/10
10035/10035 [==============================] - 13s 1ms/step - loss: 0.8218 - acc: 0.6830 - val_loss: 0.9002 - val_acc: 0.6532
Epoch 4/10
10035/10035 [==============================] - 12s 1ms/step - loss: 0.7587 - acc: 0.7107 - val_loss: 0.8985 - val_acc: 0.6550
Epoch 5/10
10035/10035 [==============================] - 13s 1ms/step - loss: 0.7038 - acc: 0.7422 - val_loss: 0.8942 - val_acc: 0.6562
Epoch 6/10
10035/10035 [==============================] - 14s 1ms/step - loss: 0.6542 - acc: 0.7624 - val_loss: 0.8975 - val_acc: 0.6583
Epoch 7/10
10035/10035 [==============================] - 13s 1ms/step - loss: 0.6147 - acc: 0.7788 - val_loss: 0.8969 - val_acc

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          400000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 165       
Total params: 720,197
Trainable params: 720,197
Non-trainable params: 0
_________________________________________________________________
